In [ ]:
import xarray as xr
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

#Define all basins of interest 
basins = {'RHINE':'6242', 'RHONE':'6243','PO':'6241', 'DANUBE':'6202', 'TITICACA':'3912', 'SANTA':'3425', 
            'OCONA':'3418', 'MAJES':'3416', 'MAGDALENA':'3227', 'AMAZON':'3203', 'YELCHO':'3429', 
            'VALDIVIA':'3428', 'SERRANO':'3426', 'RAPEL':'3423', 'PUELO':'3422', 'PASCUA':'3420', 
            'PALENA':'3419', 'HUASCO':'3412', 'COPIAPO':'3409', 'CISNES':'3408', 'BIOBIO':'3405', 'BAKER':'3404',
            'AZOPARDO':'3403', 'AISEN':'3401', 'SANTA CRUZ':'3244', 'NEGRO':'3232', 'COLORADO':'3212', 
            'CHICO':'3209', 'TORNEALVEN':'6255', 'THJORSA':'6254', 'OLFUSA':'6237', 'LULEALVEN':'6227', 
            'KUBAN':'6223', 'KALIXALVEN':'6219', 'GLOMAA':'6213', 'DRAMSELVA':'6209', 'SVARTA':'6110', 
            'LAGARFLJOT':'6104', 'JOKULSA A FJOLLUM':'6101', 'CLUTHA':'5406', 'YUKON':'4435', 'TAKU':'4431', 
             'SUSITNA':'4430','STIKINE':'4428', 'SKEENA':'4427','SKAGIT':'4426','NUSHAGAK':'4418','NASS':'4416',
            'KUSKOKWIM':'4414','FRASER':'4410', 'COPPER':'4408', 'COLUMBIA':'4406', 'ALSEK':'4401', 'NELSON':'4125', 
              'MACKENZIE':'4123','COLVILLE':'4110', 'YSYK-KOL':'2919', 'UVS NUUR':'2918', 'TARIM HE':'2914', 
              'TALAS':'2913', 'LAKE BALKHASH':'2910','HAR US NUUR':'2909', 'CHUY':'2905', 'ARAL SEA':'2902', 
              'YELLOW RIVER':'2434', 'MEKONG':'2421', 'KAMCHATKA':'2413', 'SALWEEN':'2319', 'IRRAWADDY':'2310', 
              'INDUS':'2309', 'GANGES':'2306','BRAHMAPUTRA':'2302', 'OB':'2108', 'INDIGIRKA':'2103','YANGTZE' : '2433'}

In [ ]:
#Importing all runoff data, OGGM is grouped by basin
prcp_comps = {}

for basin, ID in basins.items():
    furl = f"https://cluster.klima.uni-bremen.de/~lschuster/runs_oggm_v16/runs_2023.3/output/basins/w5e5_gcm_merged_bc_2000_2019/{ID}/components_W5E5_2000_2019_basin_{ID}.nc#mode=bytes"
    with xr.open_dataset(furl) as ds1:
        prcp_comps[basin] = ds1

In [ ]:
prcp_comps['RHONE']

In [ ]:
# Aggregating basin totals
variables_to_sum = ['liq_prcp_off_glacier', 'liq_prcp_on_glacier', 'snowfall_on_glacier', 'snowfall_off_glacier']
summed_data = {}

for basin in basins:
    # Summing the variables along the rgi_id dimension
    summed_data[basin] = prcp_comps[basin][variables_to_sum].sum(dim='rgi_id')

    # Create a new variable containing the sum of all components
    summed_data[basin]['total_precipitation'] = prcp_comps[basin]['liq_prcp_off_glacier'].sum(dim='rgi_id') + prcp_comps[basin]['liq_prcp_on_glacier'].sum(dim='rgi_id') + prcp_comps[basin]['snowfall_on_glacier'].sum(dim='rgi_id') + prcp_comps[basin]['snowfall_off_glacier'].sum(dim='rgi_id')

In [ ]:
#Calculating 2000-2020 mean annual precip
for basin in basins:
    summed_data[basin]['mean_total_precipitation'] = summed_data[basin]['total_precipitation'].mean(dim='time')

In [ ]:
precipitation_list = []

# Iterate over basins to extract the mean total precipitation
for basin in basins:
    # Extract the mean total precipitation for the current basin
    mean_total_precipitation = summed_data[basin]['mean_total_precipitation'].values.item()*1e-3
    precipitation_list.append(mean_total_precipitation)

# Create a pandas Series with basin names as index and mean total precipitation as values
precipitation_series = pd.Series(precipitation_list, index=basins)

In [ ]:
# Define the directory to save the CSV files
output_dir = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/Precipitation Analysis/'

fname = f"precip_avgs_OG.csv"

# Define the full path of the output file
output_path = os.path.join(output_dir, fname)

# Save the DataFrame as CSV
precipitation_series.to_csv(output_path, header=True, index=True)